In [1]:
import discretize as ds
import SimPEG.potential_fields as pf
from SimPEG import (
    maps, utils, simulation, inverse_problem, inversion, optimization, regularization, data_misfit, directives
)
from SimPEG.utils import io_utils
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.colors as cm
from matplotlib import rc

rc('text', usetex=True)

In [2]:
def estimate_volume(mesh, model, cutoffs, direction=1, core_region=None):
    active = np.ones(len(mesh), dtype=bool)
    if core_region is not None:
        active = np.ones(len(mesh), dtype=bool)
        for i in range(len(core_region)):
            v_min, v_max = core_region[i]
            active &= mesh.cell_centers[:, i] >= v_min
            active &= mesh.cell_centers[:, i] <= v_max
    cut_mod = model[active]
    cut_vol = mesh.cell_volumes[active]
    
    cutoffs = np.atleast_1d(cutoffs)
    dirs = np.atleast_1d(direction)
    if len(dirs) != len(cutoffs):
        if len(dirs) == 1:
            dirs = np.repeat(dirs, len(cutoffs))
    
    vols = np.zeros(len(cutoffs))
    for i in range(len(cutoffs)):
        direc = dirs[i]
        cutoff = cutoffs[i]
        if direc > 0:
            vols[i] = np.sum(cut_vol[cut_mod >= cutoff])
        else:
            vols[i] = np.sum(cut_vol[cut_mod <= cutoff])
    return vols

In [3]:
treemesh = ds.TreeMesh.readUBC('../3_blocks_model/Assets/mesh_CaMP.ubc')

In [4]:
# True 1 block model
identifier_model = treemesh.read_model_UBC( "1block_synthetic_model.ubc")

In [ ]:
#inverted model
l22_den = treemesh.read_model_UBC(repo_dir + "3_blocks_model/L2/Final_Chosen_models/minv_Tik_dw_as1.6e-05.den")
l22_den += 2.9

l22_mag = treemesh.read_model_UBC('')
l22_mag += 2.9

In [ ]:
unit1_vol = np.sum(treemesh.cell_volumes[identifier_model==-0.2])
unit2_vol = np.sum(treemesh.cell_volumes[identifier_model==0.1])
background_vol = np.sum(treemesh.cell_volumes) - unit1_vol - unit2_vol
unit1_vol, unit2_vol, background_vol